# School cafeteria violations analysis

### Overview

This project provides a concise analysis of cafeteria violations in public and private schools, aiming to contrast and compare the quantity and nature of violations in different neighborhoods to identify areas with the greatest potential risk to public health. We explore key insights, including the distribution of violation types, the prevalence of critical violations, and geographic trends. The analysis highlights the need for increased oversight and regulations in specific areas to enhance public health and safety.

### Data Understanding

The dataset, provided by the Department of Health and Mental Hygiene, comprises current inspection data for school cafeterias in NYC, covering public, private, and parochial schools. These cafeterias must adhere to food safety regulations, and inspections are conducted at least annually. The dataset includes information from the permitting process and inspection results for active school cafeterias over the past three years. It contains 7,662 violations, with 2,125 classified as critical, posing a substantial public health risk. Notably, this dataset is not suitable for historical comparisons, as it excludes ceased operations and may change due to adjudication for private for-profit schools.

### Imports

In [1]:
import pandas as pd
import numpy as np
import sqlite3 

### Data Cleaning

In [2]:
#displaying all the columns
pd.set_option('display.max_columns', None)

In [55]:
# importing data
df = pd.read_csv('/Users/macintosh/Documents/DOHMH_School_Cafeteria_inspections__2020_-_Present_.csv')
df.head()

In [56]:
#info of our data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11093 entries, 0 to 11092
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Record ID             11093 non-null  object 
 1   SchoolName            11093 non-null  object 
 2   Number                11093 non-null  object 
 3   Street                11093 non-null  object 
 4   City                  10657 non-null  object 
 5   State                 11093 non-null  object 
 6   Borough               11093 non-null  object 
 7   ZipCode               10942 non-null  float64
 8   LastInspection        11093 non-null  object 
 9   Permittee             11093 non-null  object 
 10  InspectionDate        11093 non-null  object 
 11  PTET                  11093 non-null  int64  
 12  Site Type             11093 non-null  object 
 13  Level                 7974 non-null   object 
 14  Code                  7974 non-null   object 
 15  ViolationDescriptio

The data contains 11,093 rows and 25 columns.

In [4]:
#dropping the unneeded columns
columns_to_drop = ['Permittee',
       'PTET', 'COMMUNITYBOARD', 'COUNCILDISTRICT',
    'BIN', 'BBL', 'BOROCODE']

df.drop(columns_to_drop, axis=1, inplace=True)

In [5]:
#dropping null values
df['NTA'].dropna(inplace=True)

In [6]:
#Grouping violations by date and institution 
df['Inspection event'] = df.apply(lambda row: str(row['Record ID']) + '_' + str(row['InspectionDate']), axis=1)

In [7]:
df.rename(columns={'NTA': 'NTA Code'}, inplace=True)

In [8]:
# Types of schools
count_unique = df.groupby('Site Type')['Record ID'].nunique()
count_unique

Site Type
FSE (Private School Cafeteria - For Profit)         155
FSE (Private School Cafeteria - Not-For-Profit)     473
FSE (Private School Cafeteria)                      105
FSE (Public School Cafeteria)                      1414
Name: Record ID, dtype: int64

There are four target categories of schools. For the purposes of this analysis we aggregate them into two main categories: Private and Public.

In [9]:
# Grouping all Private schools
df['School Type'] = df['Site Type'].apply(lambda x: 'Public' if 'Public' in str(x) else 'Private')

In [10]:
# Null values in Violation description indicate that no violation was found per data description.
df['ViolationDescription'] =  df['ViolationDescription'].apply(lambda x: 'no violation' if pd.isna(x) else x)

In [11]:
len(df)

11093

Importing demographics information to compare population of different neighborhoods.

In [12]:
# importing demographics table
nta_demographics_df = pd.read_excel('Demographics_by_NTA.xlsx')
nta_demographics_df.info()
nta_demographics_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 8 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Neighborhood Tabulation Area (NTA) Name  193 non-null    object 
 1   NTA Code                                 189 non-null    object 
 2   populatio                                193 non-null    int64  
 3   Latino                                   193 non-null    float64
 4   % White                                  193 non-null    float64
 5   % African American                       193 non-null    float64
 6   % Asian                                  193 non-null    float64
 7   % Other                                  193 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 12.2+ KB


,Neighborhood Tabulation Area (NTA) Name,NTA Code,populatio,Latino,% White,% African American,% Asian,% Other
0,Claremont-Bathgate,BX01,35560,60.6,1.0,37.3,0.1,0.010
1,Eastchester-Edenwald-Baychester,BX03,37887,24.6,3.1,66.0,4.2,0.022
2,Bedford Park-Fordham North,BX05,57685,75.0,4.2,15.6,3.8,0.015
3,Belmont,BX06,29115,61.0,18.2,16.3,2.1,0.024
4,Bronxdale,BX07,39423,54.9,13.9,26.1,2.9,0.022


In [13]:
# Drop null values
nta_demographics_df.dropna(inplace=True)

In [14]:
#Filtering by columns needed
nta_demographics = nta_demographics_df.copy()[['Neighborhood Tabulation Area (NTA) Name','NTA Code','populatio']]

In [15]:
nta_demographics.rename(columns={'Neighborhood Tabulation Area (NTA) Name': 'NTA Name'}, inplace=True)

In [16]:
nta_demographics.rename(columns = {'populatio': 'NTA Population'}, inplace =True)
nta_demographics.head(1)

,NTA Name,NTA Code,NTA Population
0,Claremont-Bathgate,BX01,35560


In [17]:
# Merge df and df2 on specified conditions
merged_df = df.merge(nta_demographics, on='NTA Code', how='inner')

In [18]:

merged_df['# of schools in NTA'] = merged_df.groupby('NTA Code')['Record ID'].transform('nunique')
merged_df.head(5)

,Record ID,SchoolName,Number,Street,City,State,Borough,ZipCode,LastInspection,InspectionDate,Site Type,Level,Code,ViolationDescription,LATITUDE,LONGITUDE,CENSUSTRACT,NTA Code,Inspection event,School Type,NTA Name,NTA Population,# of schools in NTA
0,40542839,"P.S. 127 MCKINLEY PARK, RALPH A. FABRIZIO SCHO...",7805,7 AVENUE,Brooklyn,NY,Brooklyn,11228.0,02/02/2023,07/28/2022,FSE (Public School Cafeteria),NaN,NaN,no violation,40.624125,-74.018826,20600.0,BK30,40542839_07/28/2022,Public,Dyker Heights,43888,9
1,50068668,PRE-K CENTER AT FORT HAMILTON PARKWAY,7415,FORT HAMILTON PARKWAY,Brooklyn,NY,Brooklyn,11228.0,12/22/2022,03/25/2022,FSE (Public School Cafeteria),G,10F,Non-food contact surface improperly constructe...,40.626151,-74.015894,20600.0,BK30,50068668_03/25/2022,Public,Dyker Heights,43888,9
2,50068671,PRE-K CENTER AT 1355 84TH STREET,1355,84 STREET,Brooklyn,NY,Brooklyn,11228.0,12/22/2022,04/04/2022,FSE (Public School Cafeteria),NaN,NaN,no violation,40.614459,-74.011425,15000.0,BK30,50068671_04/04/2022,Public,Dyker Heights,43888,9
3,50068659,PRE-K CENTER AT 1258 65TH STREET,1258,65 STREET,Brooklyn,NY,Brooklyn,11219.0,11/22/2022,11/22/2022,FSE (Public School Cafeteria),NaN,NaN,no violation,40.626547,-74.002486,19400.0,BK30,50068659_11/22/2022,Public,Dyker Heights,43888,9
4,50068671,PRE-K CENTER AT 1355 84TH STREET,1355,84 STREET,Brooklyn,NY,Brooklyn,11228.0,12/22/2022,05/10/2021,FSE (Public School Cafeteria),NaN,NaN,no violation,40.614459,-74.011425,15000.0,BK30,50068671_05/10/2021,Public,Dyker Heights,43888,9


InspectionEventCount column is created in order to know how many times each school has been inspected. 

In [19]:
merged_df['InspectionEventCount'] = merged_df.groupby('Record ID')['Inspection event'].transform('count')

In [20]:
#transforming the description to lowercase
merged_df['ViolationDescription'] = merged_df['ViolationDescription'].str.lower()

The decision is made to group all violations into 9 main categories to simplify the analysis.

In [21]:
# Creating a dictionary to Label violations categories

violations_labels_dict = {'Storage temperature': [
'Cold food item held above 41º F',
'Hot TCS food item not held at or above 140 °F',
'Cold TCS food item held above 41 °F',
'Hot food item not held at or above 140º F',
'hot tcs food item that has been cooked and  cooled is being held for service without first being reheated'],

'Food handling' : ['food not cooked to required minimum internal temperature',
'Food not protected from potential source of contamination',
'food is adulterated, contaminated',
'Food not cooled by an approved method',
'TCS food not cooled by an approved method',
'TCS food removed from cold holding',
'Thawing procedure improper',
'thawing procedures improper',
'contents are not suitable for human consumption',
'Unclean or cracked whole eggs',
'Raw food not properly washed',
'chemical or pesticide improperly stored',
'Food, supplies, and equipment not protected',
'Food, supplies, or equipment not protected',
'Canned food product observed swollen',
'Canned food product observed dented',
'damaged canned food to be returned to distributor not segregated'],

'Mice, rats, vermin': ['Mice',
'Rats',
'Live animal',
'Vermin'],

'Insects, pests': ['roaches',
'flies',
'insects',
'pests'],

'Facility': ['Plumbing not properly installed',
'Non-food contact surface improperly constructed',
'sewage disposal system',
'Sanitized equipment or utensil',
'Lighting Inadequate',
'Toilet facility not maintained',
'Toilet facility not provided',
'No facilities available to wash',
'Mechanical or natural ventilation not provided',
'mechanical or natural ventilation system not provided',
'facility not free from unsafe, hazardous, offensive or annoying condition',
'Bulb not shielded or shatterproof, in areas where there is extreme heat',
'contaminated by sewage or liquid waste',
'Failure to comply with an Order of the Board of Health',
'Lighting fixture',
'noxious gas or vapor detected'],

'Equipment issues':['Non-food contact surface or equipment',
'thermometer not provided',
'Single service item reused',
'Proper sanitization not provided',
'thermometer or thermocouple not provided',
'Food contact surface not properly washed',
'Food contact surface not properly maintained',
'Single service article reused or not protected',
'expanded polystyrene',
'maintenance of food contact surface improper',
'Cleaning and sanitizing of tableware, including dishes, utensils, and equipment deficient',
'Garbage receptacle not pest or water resistant'],

'General Hygiene': ['chemical improperly used',
'Hand washing facility not provided',
'Prohibited chemical used',
'No hand washing facility',
'Wiping cloths not stored clean',
'Wiping cloths soiled'],

'Personal Hygiene': ['worker does not wash hands thoroughly',
'eating, or drinking from open container in food preparation',
'vendor does not use utensil or other barrier to eliminate bare hand contact with food that will not receive adequate additional heat treatment',
'Personal cleanliness is inadequate',
'personal cleanliness inadequate'],

'Documents and Signs': ['Food protection certificate',
'Current valid permit',
'sign not posted',
'Establishment permit',
'Written Standard Operating Procedure',
'chemical improperly labeled',
'allergy information poster not conspicuously posted',
'acceptable manufacturer’s documentation not maintained on site',
'Document issued by the Board of Health',
'Permit not conspicuously displayed',
'nutritional information statement not posted',
'Food Protection Certificate']}


In [22]:
#transforming the dictionary to lower case 
lowercase_dict = {k: [v.lower() for v in values] for k, values in violations_labels_dict.items()}
print(lowercase_dict)

{'Storage temperature': ['cold food item held above 41º f', 'hot tcs food item not held at or above 140 °f', 'cold tcs food item held above 41 °f', 'hot food item not held at or above 140º f', 'hot tcs food item that has been cooked and  cooled is being held for service without first being reheated'], 'Food handling': ['food not cooked to required minimum internal temperature', 'food not protected from potential source of contamination', 'food is adulterated, contaminated', 'food not cooled by an approved method', 'tcs food not cooled by an approved method', 'tcs food removed from cold holding', 'thawing procedure improper', 'thawing procedures improper', 'contents are not suitable for human consumption', 'unclean or cracked whole eggs', 'raw food not properly washed', 'chemical or pesticide improperly stored', 'food, supplies, and equipment not protected', 'food, supplies, or equipment not protected', 'canned food product observed swollen', 'canned food product observed dented', 'dama

In [23]:
# Function to check if a violation description is contained in the dictionary values
def get_violation_label(description):
    if isinstance(description, float):
        return None  # Return None if the description is a float value
    for label, descriptions in lowercase_dict.items():
        if any(desc in description for desc in descriptions):
            return label
    return None




In [24]:
# Creating ViolationLabel column using our function to match violation descriptions
merged_df['ViolationLabel'] = merged_df['ViolationDescription'].apply(lambda x: get_violation_label(x))

In [26]:
len(merged_df)

10665

In [27]:
# Instantiating SQLite connection and database 
conn = sqlite3.Connection("data.sqlite")
db_path = 'school_database.db'  # Replace with the desired name of your SQLite database file
connection = sqlite3.connect(db_path)
table_name = 'schools'  # Replace with the desired table name                                                                                                                                    
merged_df.to_sql(table_name, connection, if_exists='replace', index=False) 

10665

In [28]:
connection.commit()

I saved the dataframe containing all fo the cleaned data into a separate CSV file available in the data folder of this project's repository. This file was used for visualization in Tableau.

In [ ]:
merged_df.to_csv('clean_data_labels.csv', index=False)

### Exploratory Dataset Analysis

EDA is conducted with Pandas and SQL.

In [29]:
#How many private and public schools are in our data?

query = """
SELECT `School Type`, COUNT(DISTINCT `Record ID`) count
FROM schools
GROUP BY `School Type`;
"""
pd.read_sql(query, connection)

,School Type,count
0,Private,710
1,Public,1358


In [30]:
#average number of violations per school
round(len(merged_df[merged_df['Level'].notna()])/len(merged_df['Record ID'].unique()), 2)

3.71

In [31]:
# Total count of violations
query = """
SELECT Level, COUNT(*) as ViolationCount
FROM schools
GROUP BY Level
ORDER BY ViolationCount DESC;
"""
pd.read_sql(query, connection)

,Level,ViolationCount
0,G,4738
1,None,3003
2,C,2125
3,AV,472
4,CH,186
5,CP,58
6,A,57
7,NV,26


In [32]:
# Number of unique critical and general violations

query = """
SELECT
(SELECT COUNT(DISTINCT ViolationDescription) 
FROM schools) as TotalViolations,
(SELECT COUNT(DISTINCT ViolationDescription) 
FROM schools
WHERE Level = 'C') as CriticalViolations,
(SELECT COUNT(DISTINCT ViolationDescription) 
FROM schools
WHERE Level = 'G') as GeneralViolations
;
"""
pd.read_sql(query, connection)

,TotalViolations,CriticalViolations,GeneralViolations
0,98,37,29


In [33]:
# Number violations by school type
query = """
SELECT 
       "School Type", 
       COUNT(*) as ViolationsCount
FROM schools
WHERE ViolationDescription != 'no violation'
GROUP BY "School Type"
ORDER BY ViolationsCount DESC;
"""
pd.read_sql(query, connection)


,School Type,ViolationsCount
0,Public,3848
1,Private,3814


In [34]:
#Distribution of Critical and General violation per school type
query = """
SELECT
    `School Type`,
    COUNT(ViolationDescription) as TotalViolations,
    COUNT(CASE WHEN Level = 'C' THEN ViolationDescription ELSE NULL END) as CriticalViolations,
    COUNT(CASE WHEN Level = 'G' THEN ViolationDescription ELSE NULL END) as GeneralViolations

FROM schools 
WHERE ViolationDescription != 'no violation'
GROUP BY `School Type`



ORDER BY `School Type`;

"""

pd.read_sql(query, connection)

,School Type,TotalViolations,CriticalViolations,GeneralViolations
0,Private,3814,1199,2041
1,Public,3848,926,2697


In [51]:
# Percentage of all violations by school type

query = """
SELECT
    `School Type`,
    ROUND(CAST(COUNT(CASE WHEN Level = 'C' THEN ViolationDescription ELSE NULL END) AS FLOAT) / (COUNT(*)), 2) as CriticalViolationsRatio,
    ROUND(CAST(COUNT(CASE WHEN Level = 'G' THEN ViolationDescription ELSE NULL END) AS FLOAT) / COUNT(*), 2) as GeneralViolationsRatio

FROM 

schools
WHERE ViolationDescription != 'no violation'

GROUP BY `School Type`
ORDER BY `School Type`;

"""

pd.read_sql(query, connection)

,School Type,CriticalViolationsRatio,GeneralViolationsRatio
0,Private,0.31,0.54
1,Public,0.24,0.70


In [36]:
# Average # of violations per school by school type

query = """
SELECT
    `School Type`,
    
    ROUND(CAST(COUNT(CASE WHEN Level = 'C' THEN ViolationDescription ELSE NULL END) AS FLOAT) /
    COUNT(DISTINCT "Record ID"), 2) as avg_critical,

    ROUND(CAST(COUNT(CASE WHEN Level = 'G' THEN ViolationDescription ELSE NULL END) AS FLOAT)/ 
    COUNT(DISTINCT "Record ID"), 2) as avg_general
FROM schools
WHERE Level != 'no violation'
GROUP BY `School Type`
ORDER BY `School Type`

;
"""

pd.read_sql(query, connection)



,School Type,avg_critical,avg_general
0,Private,1.93,3.29
1,Public,0.84,2.45


In [37]:
# Top 10 Critical Violations categories

query = """
SELECT DISTINCT ViolationLabel, COUNT(*) AS Count
FROM schools
WHERE Level = 'G'
GROUP BY ViolationLabel
ORDER BY Count DESC
;
"""
pd.read_sql(query, connection)

,ViolationLabel,Count
0,Facility,2222
1,Equipment issues,1309
2,"Insects, pests",476
3,"Mice, rats, vermin",353
4,Food handling,172
5,General Hygiene,121
6,Documents and Signs,85


In [38]:
# Top 10 Critical Violations types

query = """
SELECT ViolationDescription, COUNT(*) cnt_violations
FROM schools
WHERE Level = 'C'
GROUP BY ViolationDescription
ORDER BY cnt_violations DESC
LIMIT 10
;
"""
pd.read_sql(query, connection)

,ViolationDescription,cnt_violations
0,evidence of mice or live mice in establishment...,393
1,evidence of mice or live mice present in facil...,288
2,food protection certificate not held by superv...,206
3,"food contact surface not properly washed, rins...",151
4,food protection certificate (fpc) not held by ...,145
5,filth flies or food/refuse/sewage associated w...,129
6,"food, supplies, and equipment not protected fr...",111
7,food not protected from potential source of co...,104
8,cold food item held above 41º f (smoked fish a...,98
9,filth flies or food/refuse/sewage-associated (...,76


In [39]:
# What critical cafeteria violations categories are most common?
query = """
SELECT DISTINCT ViolationLabel, COUNT(*) AS Count
FROM schools
WHERE Level = 'C'
GROUP BY ViolationLabel
ORDER BY Count DESC
;
"""
pd.read_sql(query, connection)

,ViolationLabel,Count
0,"Mice, rats, vermin",691
1,Documents and Signs,352
2,"Insects, pests",307
3,Food handling,263
4,Equipment issues,247
5,Storage temperature,138
6,General Hygiene,54
7,Personal Hygiene,42
8,Facility,31


In [40]:
# How do NYC boroughs compare in number and average violations per school?

query = """
SELECT Borough, COUNT(*) AS "Violations count"
FROM schools
WHERE Level IS NOT NULL

GROUP BY Borough
ORDER BY "Violations count" DESC
LIMIT 5
;
"""
pd.read_sql(query, connection)

,Borough,Violations count
0,Brooklyn,3781
1,Queens,1560
2,Manhattan,1159
3,Bronx,931
4,Staten Island,231


In [54]:
# Average # of violations per school by Borough

query = """
SELECT
    Borough,
    ROUND(CAST(COUNT(ViolationDescription) AS FLOAT) /
    COUNT(DISTINCT "Record ID"), 1) as "Average violations",
    COUNT(DISTINCT "Record ID") as "Number of Schools"

FROM schools
WHERE ViolationDescription != 'no violation'
GROUP BY Borough
ORDER BY "Average violations" DESC

;
"""

pd.read_sql(query, connection)



,Borough,Average violations,Number of Schools
0,Brooklyn,6.0,635
1,Queens,3.9,400
2,Manhattan,3.7,312
3,Bronx,3.1,297
4,Staten Island,2.9,79


In [42]:
# of violations per neighborhood

query = """
SELECT "NTA Name", COUNT(*) AS "Critical violations"
FROM schools
WHERE "Level" = 'C'
GROUP BY "NTA Name"
ORDER BY "Critical violations" DESC
LIMIT 10;
"""
pd.read_sql(query, connection)

,NTA Name,Critical violations
0,Borough Park,250
1,Williamsburg,80
2,Bedford,61
3,Far Rockaway-Bayswater,43
4,Bensonhurst West,42
5,Flatbush,38
6,Canarsie,38
7,Prospect Lefferts Gardens-Wingate,34
8,Crown Heights South,34
9,Hudson Yards-Chelsea-Flatiron-Union Square,32


In [43]:
# Average # of violations per school by Borough

query = """
SELECT "NTA Name", COUNT(*) AS "Critical violations"
FROM schools
WHERE "Level" = 'C' AND ViolationLabel = 'Mice, rats, vermin'
GROUP BY "NTA Name"
ORDER BY "Critical violations" DESC
LIMIT 5
;
"""
pd.read_sql(query, connection)

,NTA Name,Critical violations
0,Borough Park,78
1,Williamsburg,26
2,Rugby-Remsen Village,16
3,Canarsie,15
4,Bedford,15


In [44]:
#Rank NTA neighborhoods by population next to their Mice related Critical violation count
query = """
SELECT 
    "NTA Name",
    COUNT(*) AS "Critical violations",
    RANK() OVER (ORDER BY "NTA Population" DESC) AS "Population Rank",
    "NTA Population"
FROM schools
WHERE "Level" = 'C' AND ViolationLabel = 'Mice, rats, vermin'
GROUP BY "NTA Name"
ORDER BY "Critical violations" DESC
LIMIT 5;

"""
pd.read_sql(query, connection)



,NTA Name,Critical violations,Population Rank,NTA Population
0,Borough Park,78,4,102494
1,Williamsburg,26,98,34373
2,Rugby-Remsen Village,16,47,54666
3,Canarsie,15,9,88522
4,Bedford,15,17,77847


In [45]:
#Rank NTA neoghborhoods by population next to their Mice related Critical violation count
query = """
SELECT 
    SchoolName,
    COUNT(*) AS "Mice violations count"
FROM schools
WHERE ViolationLabel = 'Mice, rats, vermin' AND "NTA Name" = 'Borough Park'
GROUP BY SchoolName
ORDER BY "Mice violations count" DESC
LIMIT 5
;

"""
pd.read_sql(query, connection)

,SchoolName,Mice violations count
0,YESHIVA TIFERETH ELIMELECH,9
1,BOBOVER YESHIVA BNEI ZION,9
2,TALMUD TORAH IMREI CHAIM,8
3,BAIS TZIPORAH SCHOOL,8
4,MOSDOS CHASIDEI SQUARE,7
